### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix



### Import Dataset

In [2]:
# Check the current working directory
%pwd

# Update the path to the CSV file
salary = pd.read_csv("../data/raw/Salaries.csv")


/var/folders/6h/k5xpmwhd71n8dl9df11lt9ph0000gn/T/ipykernel_21137/1347795926.py:5: DtypeWarning: Columns (3,4,5,6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  salary = pd.read_csv("../data/raw/Salaries.csv")


### View the Information About Dataframe

In [10]:
salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148654 entries, 0 to 148653
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Id                148654 non-null  int64  
 1   EmployeeName      148654 non-null  object 
 2   JobTitle          148654 non-null  object 
 3   BasePay           148049 non-null  object 
 4   OvertimePay       148654 non-null  object 
 5   OtherPay          148654 non-null  object 
 6   Benefits          112495 non-null  object 
 7   TotalPay          148654 non-null  float64
 8   TotalPayBenefits  148654 non-null  float64
 9   Year              148654 non-null  int64  
 10  Notes             0 non-null       float64
 11  Agency            148654 non-null  object 
 12  Status            38119 non-null   object 
dtypes: float64(3), int64(2), object(8)
memory usage: 14.7+ MB


In [11]:
salary.head()

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.0,400184.25,NaN,567595.43,567595.43,2011,NaN,San Francisco,NaN
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,NaN,538909.28,538909.28,2011,NaN,San Francisco,NaN
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,NaN,335279.91,335279.91,2011,NaN,San Francisco,NaN
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.0,56120.71,198306.9,NaN,332343.61,332343.61,2011,NaN,San Francisco,NaN
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737.0,182234.59,NaN,326373.19,326373.19,2011,NaN,San Francisco,NaN


In [13]:
# get all possible values of each column if it is a string
# for col in salary.columns:
#     if salary[col].dtype == 'object':
#         print(col, salary[col].unique())

salary["Year"].unique()
        
  

array([2011, 2012, 2013, 2014])

In [18]:
# remove leading and trailing whitespace
salary['JobTitle'] = salary['JobTitle'].str.strip()

# convert JobTitle to uppercase
salary['JobTitle'] = salary['JobTitle'].str.upper()

# remove entries with 'Not provided' job title
salary = salary[salary['JobTitle'] != 'Not provided']

# find all the unique values in the JobTitle column
job_titles = salary['JobTitle'].unique()


# sort the job titles alphabetically
job_titles.sort()

print(job_titles.size)


# fix capitalization of employee names
salary['EmployeeName'] = salary['EmployeeName'].str.title()

# convert BasePay, OvertimePay, OtherPay, Benefits, TotalPayBenefits to float
salary['BasePay'] = pd.to_numeric(salary['BasePay'], errors='coerce')
salary['OvertimePay'] = pd.to_numeric(salary['OvertimePay'], errors='coerce')
salary['OtherPay'] = pd.to_numeric(salary['OtherPay'], errors='coerce')
salary['Benefits'] = pd.to_numeric(salary['Benefits'], errors='coerce')
salary['TotalPayBenefits'] = pd.to_numeric(salary['TotalPayBenefits'], errors='coerce')

# Convert NaN in BasePay, OvertimePay, OtherPay, Benefits, TotalPayBenefits to 0
salary['BasePay'] = salary['BasePay'].fillna(0)
salary['OvertimePay'] = salary['OvertimePay'].fillna(0)
salary['OtherPay'] = salary['OtherPay'].fillna(0)
salary['Benefits'] = salary['Benefits'].fillna(0)
salary['TotalPayBenefits'] = salary['TotalPayBenefits'].fillna(0)

# find all negtive values in the TotalPayBenefits column
negative_salary = salary[salary['TotalPayBenefits'] < 0]




""" 
Negative income can have significant legal and financial implications for individuals and businesses. 
Individuals may have difficulty meeting financial obligations, accessing credit, or dealing with tax consequences. Businesses may face reporting requirements, 
tax implications, and bankruptcy considerations. It is important to address negative income promptly and seek professional advice to understand and address the 
underlying causes and consequences.
"""
# Drop the Notes column because all of the values are NaN
if 'Notes' in salary.columns:
    salary.drop('Notes', axis=1, inplace=True)

salary.info()


# * FT = Full Time, PT = Part Time

1636
<class 'pandas.core.frame.DataFrame'>
Index: 148650 entries, 0 to 148653
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Id                148650 non-null  int64  
 1   EmployeeName      148650 non-null  object 
 2   JobTitle          148650 non-null  object 
 3   BasePay           148650 non-null  float64
 4   OvertimePay       148650 non-null  float64
 5   OtherPay          148650 non-null  float64
 6   Benefits          148650 non-null  float64
 7   TotalPay          148650 non-null  float64
 8   TotalPayBenefits  148650 non-null  float64
 9   Year              148650 non-null  int64  
 10  Agency            148650 non-null  object 
 11  Status            38119 non-null   object 
dtypes: float64(6), int64(2), object(4)
memory usage: 14.7+ MB


In [8]:
for title in job_titles:
    print(title)
   

ACCOUNT CLERK
ACCOUNTANT
ACCOUNTANT I
ACCOUNTANT II
ACCOUNTANT III
ACCOUNTANT INTERN
ACCOUNTANT IV
ACPO,JUVP, JUV PROB (SFERS)
ACUPUNCTURIST
ADM, SFGH MEDICAL CENTER
ADMIN ANALYST 3
ADMIN HEARING EXAMINER
ADMINISTRATIVE ANALYST
ADMINISTRATIVE ANALYST I
ADMINISTRATIVE ANALYST II
ADMINISTRATIVE ANALYST III
ADMINISTRATIVE ENGINEER
ADMINISTRATIVE HEARING SUP
ADMINISTRATIVE SERVICES MANAGER
ADMINISTRATIVE SERVICES MGR
ADMINISTRATOR, DPH
ADMINISTRATOR, SFGH MEDICAL CENTER
ADMISSION ATTENDANT
AFFIRMATIVE ACTION SPECIALIST
AGRICULTURAL INSPECTOR
AIRPORT ASSISTANT DEPUTY DIRECTOR, BUSINESS ADMINI
AIRPORT ASSISTANT DEPUTY DIRECTOR, OPERATIONS
AIRPORT COMMUNICATIONS DISP
AIRPORT COMMUNICATIONS OFFICER
AIRPORT COMMUNICATIONS OPERATOR
AIRPORT COMMUNICATIONS SUPERVISOR
AIRPORT ECONOMIC PLANNER
AIRPORT ELECTRICIAN
AIRPORT ELECTRICIAN SUPERVISOR
AIRPORT EMERG PLANNING COORD
AIRPORT EMERGENCY PLANNING COORDINATOR
AIRPORT MECHANICAL MAINT SPRV
AIRPORT MECHANICAL MAINTENANCE SUPERVISOR
AIRPORT NOISE ABAT

In [15]:
# How have salaries changed over time between different groups of people?
# Convert 'BasePay' to numeric, coercing errors to NaN
salary['BasePay'] = pd.to_numeric(salary['BasePay'], errors='coerce')

# group by year and title
grouped = salary.groupby(['JobTitle', 'Year'])
# find the mean of the base pay for each group
grouped = grouped['BasePay'].mean()

# print the grouped data
grouped.head()


    


JobTitle                     Year
ACCOUNT CLERK                2011    43300.806506
ACCOUNTANT                   2011    46643.172000
ACCOUNTANT INTERN            2011    28732.663958
ACPO,JuvP, Juv Prob (SFERS)  2014    62290.780000
ACUPUNCTURIST                2011    66374.400000
Name: BasePay, dtype: float64